# Read outputs of analyze_functionalized for further custom analysis

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import sys
sys.path.append('ext')
import plot_util
import accelerometers

In [ ]:
inputdir = pathlib.Path.cwd() / 'outputdata'
# inputdir is previous outputdir
outputdir = pathlib.Path.cwd() / 'outputdata' / 'explore'
outputdir.mkdir(exist_ok=True)

pathlib.Path.cwd()

In [ ]:
combined = pd.read_csv(inputdir / 'Combined.csv', header=[0, 1], index_col=0)
combined.head()

In [ ]:
combined.reset_index()

In [ ]:
# Print number of trips by category
print('Number of Trips')
combined.groupby('Category', axis=1).count().T[combined.index[0]]

In [ ]:
# Recreate standard combined average plot
combined_bycat_log = combined.map(np.log10).groupby('Category', axis=1).mean()
plot_util.generic_plot(combined_bycat_log.rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='g$^2$/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode',
                       logx=True, logy=True)

In [ ]:
# Plot all trips for each mode
for cat in combined.columns.levels[0]:
    print(cat)
    plt.figure()
    data = combined[cat].apply(np.log10).rename(np.log10, axis=0)
    plot_util.generic_plot(data, kind='line', xlabel='Hz', ylabel='$g^2$/Hz',
                       output_directory=outputdir, title='Power Spectra for all Trips, ' + cat,
                       logx=True, logy=True)
    plt.close()

From here down may need customization

In [ ]:
# Look at cum rms
cum_rms = combined_bycat_log.apply(lambda xx: (10**np.interp(fp=xx, xp=xx.index, x=np.arange(0.01, 30, 0.01))).cumsum()**0.5)

In [ ]:
cum_rms_norm = cum_rms.apply(lambda x: x / x.max())
cum_rms_norm = cum_rms_norm.rename(lambda x: np.log10(x * 0.01))
plot_util.generic_plot(cum_rms_norm, 'line', output_directory=outputdir, title='Cum RMS', xlabel='Hz', logx=True)